In [ ]:
## matplotlib 사용시 한국어 나오게 하는 코드

!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

font = fm.FontProperties(fname=fontpath, size=9)

#fm._rebuild()

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
import pandas as pd #판다스 패키지 불러오기
import lightgbm as lgb #트리 기반 학습 알고리즘
import folium #파이썬 지도를 다루는 라이브러리

In [ ]:
##데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission_제출양식.csv')

In [ ]:
train.head()
#id : 고유 id
#date : 버스 운행 날짜
#route_id : 버스 노선 ID
#vh_id : 버스 id
#route_nm : 버스 노선 실제 번호
#now_latitude : 현재 정류소의 위도
#now_longitude : 현재 정류소의 경도
#now_station : 현재 정류소 이름
#now_arrive_time : 현재 정류장에 도착한 시간
#distance : 현재 정류장에서 다음 정류장까지 실제 이동한 거리
#next_station : 다음 정류소 이름
#next_latitude : 다음 정류소의 위도
#next_longitude : 다음 정류소의 경도
#next_arrive_time : 다음 정류장에 도착할 때 까지 걸린 시간(단위:초)으로 답안 제출을 위해서 예측해야 되는 값

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


In [ ]:
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [ ]:
submission.head()

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0


In [ ]:
print(train.shape)
print(test.shape)
print(submission.shape)

(210457, 14)
(91774, 13)
(91774, 2)


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


In [ ]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                91774 non-null  int64
 1   next_arrive_time  91774 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB


In [ ]:
train[['route_id','vh_id']].drop_duplicates() #중복되는 값 줄이기

,route_id,vh_id
0,405136001,7997025
181,405136001,7997034
381,405136001,7997035
539,405136001,7997040
717,405136001,7997044
...,...,...
20226,405136522,7997004
26037,405320116,7983225
27098,405320122,7983067
45506,405328102,7983401


In [ ]:
train[['route_id','vh_id']].drop_duplicates().groupby('route_id').count()#중복되는 값 없앤 상태에서 구룹화하기기

,vh_id
route_id,
405136001,10
405136002,5
405136007,1
405136012,3
405136521,6
405136522,15
405320111,3
405320112,5
405320113,1


In [ ]:
train[['route_id','next_arrive_time']].drop_duplicates().groupby('route_id').mean()
#버스 도착시간이 상이함을 확인하기

,next_arrive_time
route_id,
405136001,214.493827
405136002,494.006873
405136007,151.703125
405136012,521.729231
405136521,596.943262
405136522,439.278846
405320111,206.722543
405320112,228.140704
405320113,163.738462


In [ ]:
##지도 시각화하기
coord = [35.1559361, 129.1384361]
map_osm = folium.Map(location = coord)
map_osm

In [ ]:
map_osm = folium.Map(location=coord, zoom_start=12)
map_osm

In [ ]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Terrain')
map_osm

In [ ]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Toner') #Stamen Toner 흑백
map_osm

In [ ]:
coord = [35.2335123, 129.0810047]

map_osm = folium.Map(location=coord, zoom_start=12) #zoom_start 확대

folium.Marker(coord, popup='부산대학교', tooltip = '부산대학교 tooltip').add_to(map_osm) # 마커 생성 뒤 지도에 추가해주는 기능

map_osm.save('index.html')

map_osm

In [ ]:
#Series 내에 문자열을 일괄적으로 slicing
train['now_arrive_time']

0         06시
1         06시
2         06시
3         06시
4         07시
         ... 
210452    21시
210453    21시
210454    21시
210455    21시
210456    21시
Name: now_arrive_time, Length: 210457, dtype: object

In [ ]:
train['now_arrive_time'].str.slice(stop = -1) #시 빠짐짐

0         06
1         06
2         06
3         06
4         07
          ..
210452    21
210453    21
210454    21
210455    21
210456    21
Name: now_arrive_time, Length: 210457, dtype: object

In [ ]:
train['now_arrive_time'].str.slice(stop = -1).astype('int')#

0          6
1          6
2          6
3          6
4          7
          ..
210452    21
210453    21
210454    21
210455    21
210456    21
Name: now_arrive_time, Length: 210457, dtype: int64

In [ ]:
train['now_arrive_time'] = train['now_arrive_time'].str.slice(stop = -1).astype(int)

In [ ]:
train.info()  #t사용할 수 있는 데이터 형태로 바뀜

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 22.5+ MB


In [ ]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,6,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,6,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,6,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),6,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,7,374.0,은남동,33.485822,126.490897,64


In [ ]:
test['now_arrive_time'] = test['now_arrive_time'].str.slice(stop = -1).astype(int)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 22.5+ MB


In [ ]:
##모델 구축
features = ['now_latitude', 'now_longitude', 'now_arrive_time', 'distance']
target = ['next_arrive_time']

In [ ]:
X_train, X_test, y_train = train[features], test[features], train[target]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
model_dict = {
    'linear':LinearRegression(), #선형회귀귀
    'rf': RandomForestRegressor(random_state=0, n_jobs=-1), #랜덤포레스트트
    'lgbm' : lgb.LGBMRegressor(random_state=0) #트리기반 학습 알고리즘즘
}

In [ ]:
model_dict.keys()

dict_keys(['linear', 'rf', 'lgbm'])

In [ ]:
model_dict['linear']

LinearRegression()

In [ ]:
model_result = {}

for key in model_dict.keys():
  print('#### 훈련중 ####')
  model_dict[key].fit(X_train, y_train)
  print('#### 예측 중 ####')
  model_result[key] = model_dict[key].predict(X_test)

#### 훈련중 ####
#### 예측 중 ####
#### 훈련중 ####


<ipython-input-46-bc360dde24ba>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dict[key].fit(X_train, y_train)


#### 예측 중 ####
#### 훈련중 ####
#### 예측 중 ####


In [ ]:
model_result['rf']

array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
        31.84467172,  88.46184286])

In [ ]:
model_result['lgbm']

array([ 42.12400643,  60.24878305, 120.81241189, ..., 105.81387439,
        30.82952485, 111.66787953])

In [ ]:
lr_submit= submission.copy()
rf_submit= submission.copy()
lgbm_submit= submission.copy()

In [ ]:
lr_submit['next_arrive_time'] = model_result['linear']
rf_submit['next_arrive_time'] = model_result['rf']
lgbm_submit['next_arrive_time'] = model_result['lgbm']

In [ ]:
lgbm_submit.head()

,id,next_arrive_time
0,210457,42.124006
1,210458,60.248783
2,210459,120.812412
3,210460,61.146493
4,210461,150.454103


In [ ]:
lr_submit.to_csv('lr_submit.csv', index = False)
rf_submit.to_csv('rf_submit.csv', index = False)
lgbm_submit.to_csv('lgbm_submit.csv', index = False)